# Import Libraries

In [1]:
import pystac_client
import planetary_computer
import geopandas as gpd

# Make an AOI point and buffer - view on map

In [2]:
from shapely.geometry import shape
from shapely.ops import transform

loc = {"type": "Point", "coordinates": [-105.3090, 40.6554]}

geom = shape(loc)
geom_buff=gpd.GeoSeries(geom.buffer(0.01),crs='EPSG:4326')
geom_buff.explore()

# Open Catalog and search for dataset in collection

In [3]:
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=planetary_computer.sign_inplace)

# Collections for LIDAR
#"3dep-lidar-hag",
#"3dep-lidar-dsm",
#"3dep-lidar-pointsourceid",
#"3dep-lidar-intensity",
#"3dep-lidar-dtm",
#"3dep-lidar-dtm-native",
#"3dep-lidar-returns",
#"3dep-lidar-classification"

srch_hag = catalog.search(collections=["3dep-lidar-hag"], intersects=geom.buffer(0.01))
ic_hag = srch_hag.get_all_items()
df_hag = gpd.GeoDataFrame.from_features(ic_hag.to_dict(), crs="epsg:4326")
display(df_hag.head(5))
p1=geom_buff.explore(color='orange')
p2=df_hag.explore(m=p1,color='blue')
p2

C:\Users\magst\anaconda3\envs\ee\Lib\site-packages\pystac_client\item_search.py:850: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


,geometry,datetime,proj:epsg,proj:shape,3dep:usgs_id,end_datetime,raster:bands,proj:geometry,proj:projjson,proj:transform,start_datetime
0,"POLYGON ((-105.38661 40.70634, -105.38619 40.6...",None,None,"[4097, 4097]",USGS_LPC_CO_SoPlatteRiver_Lot2a_2013_LAS_2015,2014-05-31T00:00:00Z,"{'unit': 'metre', 'nodata': 'nan', 'sampling':...","{'type': 'Polygon', 'coordinates': [[[-105.386...",{'name': 'NAD83 / UTM zone 13N + NAVD88 height...,"[2.0, 0.0, 467342.0, 0.0, -2.0, 4506230.0, 0, ...",2013-10-25T00:00:00Z


# Export tif to output_tif_path

In [ ]:
ic_hag[0].assets['data'].href

import requests

# Select the STAC item you want to export
item = ic_hag[0]  # Replace 0 with the index of the item you want to export

# Get the download URL of the 'data' asset
asset_url = item.assets['data'].href

# Define the output GeoTIFF file path
output_tif_path = "C:\\Users\\output\\path\\imagename.tif"

# Download the asset
response = requests.get(asset_url)

# Check if the download was successful
if response.status_code == 200:
    with open(output_tif_path, 'wb') as output_file:
        output_file.write(response.content)
    print(f"GeoTIFF file '{output_tif_path}' has been downloaded.")
else:
    print(f"Failed to download the asset.")